In [1]:
# -*- coding: utf-8 -*-
import os
import sys
import urllib2
import pandas as pd
import re
from bs4 import BeautifulSoup
import StringIO
import time

In [2]:
# Set saved file path
file_folder_path = '/Users/maeng-dev/Documents/GitHub/Python_Lectures/2018_02_SMU_Lecture_Slide/ch09/code/csv/'

# Set the sleep time
sleep_time_sec = 1

# Make a csv file each page  1 page have 30 question & Answer pairs
csv_iteration_num = 1 # question & Answer pairs

### if you have a problem during running this crawler use this parameters
# Loan 0 Banking 1 Creadit card 2 Insurance 3 Travel 4 Study 5 Computer 6 Developer 7 Phone_qna 8
# when you select no.2 it will be automatically begin crawling include 3,4,5,..
select_category = 3
select_start_page_no = 0 # if your last file name like this 'Loan_4~6_kor_forum.csv' please put the value as '6'

In [3]:
# Target web page
# Section Name
#  - Insurance 47,641 Questions ||  http://www.ppomppu.co.kr/zboard/zboard.php?id=consulting&page=1&category=1&divpage=11
#  - Credit card 167,457 Questions  ||  http://www.ppomppu.co.kr/zboard/zboard.php?id=card_consult&page=1&divpage=35
#  - Banking 10,235 Questions ||  http://www.ppomppu.co.kr/zboard/zboard.php?id=money&page=1&category=8&search_type=sub_memo&keyword=%C0%BA%C7%E0&divpage=47
#  - Loan 757 Questions ||  http://www.ppomppu.co.kr/zboard/zboard.php?id=loan_consult&page=1&divpage=1

# http://www.ppomppu.co.kr/zboard/view.php?id=money&page=1&divpage=47&category=8&search_type=sub_memo&no=262147
# http://www.ppomppu.co.kr/zboard/view.php?id=money&amp;page=1&amp;divpage=47&amp;category=8&amp;search_type=sub_memo&amp;keyword=%EC%9D%80%ED%96%89&amp;no=262174
# 

In [4]:
def url_list_per_page(pre_url, min_page, max_page, post_url):
    url_list = []
    for i in range(min_page, max_page+1):
        url_list.append(pre_url+str(i)+post_url)
    return url_list

In [5]:
# List Page Url Patterns
# If your crawling work is interrupted by unexpected accident, you can modify the number of parameters to resume your crawling
insurance_urls = url_list_per_page('http://www.ppomppu.co.kr/zboard/zboard.php?id=consulting&page=', 1, 1591,'&category=1&divpage=11')
credit_urls = url_list_per_page('http://www.ppomppu.co.kr/zboard/zboard.php?id=card_consult&page=', 1, 5620 ,'&divpage=35' )
banking_urls = url_list_per_page('http://www.ppomppu.co.kr/zboard/zboard.php?id=money&page=', 1, 513 , '&category=8&search_type=sub_memo&keyword=은행&divpage=4')
loan_urls = url_list_per_page('http://www.ppomppu.co.kr/zboard/zboard.php?id=loan_consult&page=' , 1, 27,'&divpage=1' )
travel_urls = url_list_per_page('http://www.ppomppu.co.kr/zboard/zboard.php?id=tour&page=', 1, 2476, '&divpage=1')
study_urls = url_list_per_page('http://www.ppomppu.co.kr/zboard/zboard.php?id=study&page=',1, 1790, '&divpage=1')
com_urls = url_list_per_page('http://www.ppomppu.co.kr/zboard/zboard.php?id=computer&page=',1, 14042,'&divpage=1')
dev_urls = url_list_per_page('http://www.ppomppu.co.kr/zboard/zboard.php?id=developer&page=',1, 589 , '&divpage=1')
phone_qna_urls = url_list_per_page('http://www.ppomppu.co.kr/zboard/zboard.php?id=phone2&page=', 1, 23772, '&category=7&divpage=1')

# Contents Urls Patterns
insurance_con_url = 'http://www.ppomppu.co.kr/zboard/view.php?id=consulting&page=1&divpage=11&category=1&no='
credit_con_url = 'http://www.ppomppu.co.kr/zboard/view.php?id=card_consult&page=1&divpage=35&no='
banking_con_url = 'http://www.ppomppu.co.kr/zboard/view.php?id=money&page=1&divpage=47&category=8&search_type=sub_memo&no='
loan_con_url= 'http://www.ppomppu.co.kr/zboard/view.php?id=loan_consult&page=1&divpage=1&no='
travel_con_url ='http://www.ppomppu.co.kr/zboard/view.php?id=tour&page=2&divpage=1&no='
study_con_url = 'http://www.ppomppu.co.kr/zboard/view.php?id=study&page=1&divpage=17&no='
com_con_url = 'http://www.ppomppu.co.kr/zboard/view.php?id=computer&page=2&divpage=93&no='
dev_con_url = 'http://www.ppomppu.co.kr/zboard/view.php?id=developer&page=2&divpage=4&no='
phone_qna_con_url = 'http://www.ppomppu.co.kr/zboard/view.php?id=phone2&page=2&divpage=302&category=7&no='
# for j in loan_urls:
#     print j

# parameter control
cate_p = [
            ['Loan', loan_urls, loan_con_url, 27], #0
            ['Banking',banking_urls, banking_con_url, 513], #1
            ['Credit card', credit_urls, credit_con_url, 5620], #2 
            ['Insurance',insurance_urls, insurance_con_url, 1591], #3 
            ['Travel',travel_urls, travel_con_url, 2476], #4
            ['Study',study_urls, study_con_url, 1790], #5
            ['Computer',com_urls, com_con_url, 14042], #6
            ['Dev',dev_urls, dev_con_url, 589], #7
            ['Phone_qna', phone_qna_urls, phone_qna_con_url, 23772], #8
         ]

In [7]:
### Remove special character & duplicated spaces
def special_symbol_remover(target_text):
    # exception_regex = '[%$&+]' ## Exception case: It can prononciate in Korean
    sym_regex = re.compile(ur'[^0-9a-zA-Z가-힣%$&+. ]+') ## Remove symbols without space  # symbols -> space [ ] 
    ex_decimal_point_regex = ur'[0-9][\.][0-9]+' ## Decimal point & url 
    space_regex = re.compile(ur'[ ]{2,}') ## two more spaces [ ][ ] -> one space[ ]
    dot_regex = re.compile(ur'[\.]{2,}') ## two more dots ... -> one space[ ]
    while target_text[len(target_text)-1] == '.': # whlie loop can handle multiple dots at the end of sentense 
        target_text = target_text[0:len(target_text)-1]
## Use this block for exception
#     if re.search(ex_decimal_point_regex, target_text):
#         result_text = target_text
#     elif re.search(ex_decimal_point_regex, target_text):
#         result_text = target_text
    sp_tuple = re.subn(sym_regex, r" ", target_text) # Special Characters
    dot_tuple = re.subn(dot_regex, r" ", sp_tuple[0]) # Duplicated dots remover
    space_tuple = re.subn(space_regex, r" ", dot_tuple[0]) # Duplicated Space remover
    result_text = space_tuple[0]
    return result_text 
# print special_symbol_remover('오늘 한국 증시는 30% 포인트 상승했습니다......'.decode('euc-kr').encode('utf-8'))
# print special_symbol_remover(u'오늘 금리는...    어떤가요?  ^^')
# print special_symbol_remover(u'금일 금리는 0.35p 상승하는 데 그쳤습니다...')

In [8]:
def tag_remover(target_text):
    tag_list = []
    tag_list.append(ur'<textarea class="ori_comment">')
    tag_list.append(ur'</textarea>')
    for tag in tag_list:
        target_text = re.sub(tag, '', target_text)
    return target_text

In [9]:
### parameter control 
for j in range (select_category, len(cate_p)):
    df = pd.DataFrame(columns=('Section', 'No', 'Question', 'List URL', 'Contents URL', 'Answer1', 'Answer2', 'Answer3', 'Answer4', 'Answer5'))
    category = cate_p[j][0]
    list_url_n = cate_p[j][1]
    con_url_patterns = cate_p[j][2] # To add prefix url 
    last_page_n = cate_p[j][3]
#     print category_name, url_list, '|    | ', content_url_patterns
#     category = 'banking'
    page_counter = 1 # start
    for k in list_url_n[select_start_page_no: ]:
        list_url = k
        print 'go to questions list page '+str(page_counter)
        data = urllib2.urlopen(list_url).read().decode('euc-kr', errors='ignore')
        time.sleep(sleep_time_sec) 
        soup = BeautifulSoup(data, 'html.parser')
        # CSS Selecter 
        #revolution_main_table > tbody > tr:nth-child(15) > td:nth-child(4) > a > font
        p_title = soup.select ('tr > td > a ')
        # find contents unique no
        p = re.compile("(&no=)[0-9]+")
        for i in p_title:
            for m in p.finditer(i.get('href')):
        #         print special_symbol_remover(i.text) # title
                cont_no = str(i.get('href')[m.start()+4:])
                temp_link = con_url_patterns+cont_no
                print temp_link # content link
                time.sleep(sleep_time_sec) 
                content = urllib2.urlopen(temp_link).read().decode('euc-kr', errors='ignore')
                temp_soup = BeautifulSoup(content, 'html.parser')
                selected_comments = temp_soup.select('tr > td > div')
                comments = []
                for cmt in selected_comments: 
                    temp_soup = BeautifulSoup(str(cmt), 'html.parser')
                    ## Remove repeated comments 
                    if temp_soup.textarea is None: 
                        # print temp_soup.get_text()
                        comments.append(temp_soup.get_text().replace('\r', ' ').replace('\n', ' ').replace('  ', ' '))
                    else: 
                        pass
                try:
                    # one_line_comment = special_symbol_remover(comments[0].text.splitlines()[0]) # read one line of comments
                    # one_line_comment = special_symbol_remover(comments[0].text.splitlines()[0]) # read one line of comments
                    q=0
                    temp_comments = []
                    for u in comments:
                        temp_comments.append(special_symbol_remover(u))
                        q += 1
                except IndexError: 
                    pass
                    # temp_comments[0] = '#NULL'
                ## question multiple lines
                ## body > div.wrapper > div.contents > div.container > div > table:nth-child(15) > tbody > tr:nth-child(1) > td > table > tbody > tr > td > table > tbody > tr > td > p:nth-child(2)
                ## </p><!--"&lt;--></td></tr></table><!--/DCM_BODY--></td></tr></tbody></table>
                temp_soup2 = BeautifulSoup(content, 'html.parser')
                ## body > div.wrapper > div.contents > div.container > div > table:nth-child(15) > tbody > tr:nth-child(1) > td > table > tbody > tr > td > table > tbody > tr > td
                selected_questions = temp_soup2.select('tbody > tr > td')
                try:
                    question = ''
                    for sq in selected_questions: 
                        # Quetion = title + content
                        question = question + i.text + sq.text
                        # Remove newline and repeated spaces
                        question = question.replace('\r', ' ').replace('\n', ' ').replace('  ', ' ')
                except: 
                    question = '#NULL'
                print question
                answer = []
                for h in range(0,5):
                    try:
                        answer.append(temp_comments[h])
                    except IndexError:
                        answer.append('#NULL')
                df = df.append( {'Section':category, 'No' : cont_no, 'Question': question, 'List URL' : list_url, 'Contents URL' : temp_link, 'Answer1' : answer[0], 'Answer2': answer[1], 'Answer3': answer[2], 'Answer4': answer[3], 'Answer5': answer[4] }, ignore_index=True)
                #commentContent_1460166
        if page_counter % csv_iteration_num == 0: 
            csv_name = file_folder_path+category+"_"+str(page_counter-csv_iteration_num+1+select_start_page_no)+"~"+str(page_counter+select_start_page_no)+"_kor_forum.csv"
            df.to_csv(csv_name, encoding='utf-8')
            print "Saved at"+csv_name
            df = pd.DataFrame(columns=('Section', 'No', 'Question', 'List URL', 'Contents URL', 'Answer1', 'Answer2', 'Answer3', 'Answer4', 'Answer5'))
        elif page_counter == last_page_n:
            csv_name = file_folder_path+category+"_"+str(page_counter-csv_iteration_num+1+select_start_page_no)+"~"+str(page_counter+select_start_page_no)+"_kor_forum.csv"
            df.to_csv(csv_name, encoding='utf-8')
            df = pd.DataFrame(columns=('Section', 'No', 'Question', 'List URL', 'Contents URL', 'Answer1', 'Answer2', 'Answer3', 'Answer4', 'Answer5'))
            print "Saved at"+csv_name
        else: 
            pass
        page_counter += 1

go to questions list page 1
http://www.ppomppu.co.kr/zboard/view.php?id=consulting&page=1&divpage=11&category=1&no=6

http://www.ppomppu.co.kr/zboard/view.php?id=consulting&page=1&divpage=11&category=1&no=2509
   보험상담실 이용안내 및 인증컨설턴트 소개  원하시는 설계를 뽐뿌 컨설턴트들과 함께 만들수 있습니다.  (＊인증설계사를 통해 정확한 설명과 제대로 된 가입절차 안내를 받으실 수 있습니다.)  ">   보험상담실 이용안내 및 인증컨설턴트 소개  1 2 
http://www.ppomppu.co.kr/zboard/view.php?id=consulting&page=1&divpage=11&category=1&no=74236
[꼭 가입] 91년생 남자 암보험 설계 부탁 드립니다. [꼭 가입]  *판매사의 수익이 아닌, 질문자의 입장에서, 암보험 설계 해주시는 분에게 꼭 가입하려 합니다.* 91년생 남자입니다. 1. 어릴적 폐 결핵 앓은적이 있습니다. 2. 선천적으로 장이 좋지 않아 장염때문에 병원을 자주 다닙니다. 3. 할아버지가 위암으로 돌아가셨습니다. 4. 보장형 / 환급형 중에 고민 하고 있습니다. (환급형은 보험사의 수익이 많이 나는 구조임을 알고 있습니다.) 위 4가지 사항을 잘 고려하여, 질문자의 입장에서 보험 설계 해주시는분께 무조건 가입할 예정입니다. 가족 보험 설계하듯 특약 끼워넣기 없이 정성어린 설계 해주실분 답변 부탁드립니다. (필요한 것들만 넣으려 합니다.)
http://www.ppomppu.co.kr/zboard/view.php?id=consulting&page=1&divpage=11&category=1&no=74235
남편 암보험 문의  남편이 실비만 있고 다른 보험이 없어서 가입하려고 계획중입니다 86년 2월생, 회사 일반사무직입니다 잡다한거 없이 암, 심혈관질환, 허혈성

운전자보험 만기예요  [안내사항] ① 설계와 가입문의는 아래 URL의 인증컨설턴트 이미지를 클릭하여 문의 하시기 바랍니다. - 관련 URL: http://www.ppomppu.co.kr/zboard/view.php?id=consulting&no=2509 ② 빠른 설계와 가입문의를 원하시는 회원님들께서는 본게시판의 상담신청 배너를 이용 하시기 바랍니다. - 게시글의 열람은 누구나 가능하기 때문에 개인정보는 절대 기재하지 마십시요. (이름/주민번호/연락처/이메일등) ------------------------------------------<게시물 작성 시 위 내용은 삭제하고 작성하세요!>--------------------------------------------   운전자보험에 가입하고자 합니다 가장 저렴한 보험으로 하고 싶은데 상담 부탁드립니다 쪽지 부탁드려요
http://www.ppomppu.co.kr/zboard/view.php?id=consulting&page=1&divpage=11&category=1&no=74212
보험가입문의드립니다, 30세(88년10월생) 여성입니다. 30세라 어린이 보험 가입이 가능하다고 들어서 설계 부탁드립니다. 실비는 가지고 있구요, 암 및 2대질환 보험을 가입하려합니다.   원하는 조건은 1) 진단비 암3,000만원이상, 뇌혈관/허혈성심장질환 각각 2,000만원 이상2) 질병수술비/골절진단금/질병후유장해/일생생활배상책임 항목도 고액이 아니라면 갖추고 싶습니다.   비혼예정이라 질병사망이나 상해사망 같은 조항은 없거나 기본으로 들어가야 한다면 최소금액이었으면 좋겠습니다. (보험료납입면제 조항이 같이 있었으면 좋겠습니다)보험가입문의드립니다, 1 2 
http://www.ppomppu.co.kr/zboard/view.php?id=consulting&page=1&divpage=11&category=1&no=74211
만 24세 보험 문의  현재 1994년생 24세 입니다.   대학생이며, 현재 미국에 잠시 있는데 크리스마스 전에

보험사끼리 심사내역 공유하나요?  보험가입심사를 넣으면 승인심사거절당한 내역을 보험사끼리 공유할까요? 어떤 이유로 거절되었는에대한 내역이여 * insurance 에서 게시물 이동되었습니다 (2018-11-12 23:10 / no : 72045)
http://www.ppomppu.co.kr/zboard/view.php?id=consulting&page=1&divpage=11&category=1&no=74190
암보험 질문요  제가 메리츠에 암이랑 롯데보험에 천만원 암보험이있는데 일본생활오래하고 술담배많이해서 암보험을 하나더 가입하고 싶은데(3000만~5000만)어떻게 가입하는게 좋을까요?추천하는회사가있을까요?괜찮으면 가입설계서 받아보고싶어요 연락주세요 * insurance 에서 게시물 이동되었습니다 (2018-11-12 23:06 / no : 72046)
http://www.ppomppu.co.kr/zboard/view.php?id=consulting&page=1&divpage=11&category=1&no=74189
보험가입 31세 여성 종합암보험 가입문의드립니다보험가입 1 2 
http://www.ppomppu.co.kr/zboard/view.php?id=consulting&page=1&divpage=11&category=1&no=74188
사망정기보험 문의해요  현재 76년생남자이구요 65세까지보장받는 소멸성으로 사망정기보험 문의드려요. 3억원보장원합니다. 온라인이 제일싸다들었어요.  비흡연자입니다. 사무직.76년 11원생입니다 5년이내 병력없습니다 제일저렴한 회사와 월납입금액좀 알려주세요
http://www.ppomppu.co.kr/zboard/view.php?id=consulting&page=1&divpage=11&category=1&no=74187
29살 남자입니다 암보험 설계문의드립니다 현재 메리츠 실비보험 월 12800원씩 내고있습니다 암보험 가입하려고하는데 비갱신형 미환급 으로 알아보고있습니다 보험 설계 부탁드려요!29살 남자입니다 암보험 설계문

만기가 몇년 안남은 종신보험  안녕하세요 만기가 몇년 남지않은 종신보험이 있는데요 갱신형특약이 유일하게 두개가 있는데 다들 갱신형은 빼야한다고 해서요 약관을 보니 분명 우리나라말이긴한데 무슨 말인지 도무지 모르겠어어 ㅠㅠ 갱신은 둘 다 5년단위에요 빼도 무관할가요? n * insurance 에서 게시물 이동되었습니다 (2018-11-12 15:42 / no : 72030)
http://www.ppomppu.co.kr/zboard/view.php?id=consulting&page=1&divpage=11&category=1&no=74167
자동차보험 문의드립니다ㅜㅜ  만 29세, 여자 YF소나타 2011년식   16.11.11 중고차 구매 후 현재까지 삼성다이렉트 가입 중이었는데 이번에 갱신하려고 하니 자꾸 앱에서 시스템오류라고 조금있다가 해보라고 해서 ㅜㅜ 당연히 되는줄 알고 있다가 오늘 만료 후 거절당한 케이스입니다.. 1년동안 사고처리는 3회 정도인데 이제와서 알아보니 모든 보험사 다이렉트는 다 거절이고.. 설계사 끼고 하면 된다는데 어찌해야 하는지 모르겠어요ㅠㅠ 도와주세요ㅠㅠ (1= 주차장에서 앞휀다 보험으로 고침, 2= 주차장에서 뒤범퍼 & 라이트 깨져 보험으로 고침, 3= 주차장에서 상대 차 후방추돌로 보험처리)  자꾸 프로그램 오류라고 해서 당연히 되겠지 생각하고 걱정도 안했는데 .. 하필 저번주에 속초로 놀러가는 바람에 전화연락도 안되고.. 걱정도 안하고 있었는데.. ㅠㅠ 오늘 갑자기 전화문의하니 안된다고하네요ㅠㅠ 이미 보험은 끝났어요ㅠㅠ 도와주세요ㅠㅠ
http://www.ppomppu.co.kr/zboard/view.php?id=consulting&page=1&divpage=11&category=1&no=74166
실손보험 단독  자녀 실비만 단독으로 들고싶어서 알아보고있습니다 혹시 실비는 어디가 더 좋고 나쁘고가 있는지, 어느 회사가 가장 좋은지 궁금합니다 ㅎ 그럼 수고하세요!
http://www.ppomppu.co.kr/zboard/view.p

실손의료보험 인상률  실비가입하려하는데 농협이인상률이 제일 낮은데 그럼 소비자입장에선 인상률낮은게 제일좋은게 아닌가요? 몇몇분께 실비말고도 보험설계를 받았는데 영 못믿겠네요. 저렴하게 그리고 가성비좋고 진실되게 설계받아보고싶네요 그외는 정중히 사양합니다 실비추천과 3대질병진단비(암뇌심장) 추천부탁합니다
http://www.ppomppu.co.kr/zboard/view.php?id=consulting&page=1&divpage=11&category=1&no=74140
실손보험 문의드려요  현재 만49세 남자 사무직 병력:없음 x 고혈압 당뇨 수술이력 없습니다 x 복용약없습니다 x 실손보험 매달 나가는금액입니다 적립식은 아니고 소멸입니다 보험료가 매달 현재보다 저렴하고  더좋은 실손보험 있다면 추천부탁드립니다  현재 까지 보험들고 진료비 및 약값 5만원내외 받아본게 전부 입니다 
http://www.ppomppu.co.kr/zboard/view.php?id=consulting&page=1&divpage=11&category=1&no=74139
(유지/해지 고민) 삼성생명 생활자금 받는 변액보험 이거 괜찮나요...?  안녕하세요? 저는 24살  여대생입니다.   저는 현재 농협 다솜플러스 보험을 1년째 유지중이구요 최근에 어머니가 제 의사는 묻지 않으시고 삼성생명 생활자금 받는 변액유니버설종신1.0無_표준 이 보험을 가입시키셨어요... 지인 추천으로요     월 13만 5천원 정도 나가고 있고  현재 5개월 정도 70~80만원 나간 상황이예요     근데 제 계좌로 나가서  저는 학생이라 아직 일정한 수입도 없는데  너무 부담돼서 해지하고 싶거든요 ㅠㅠ......     근데 5개월동안 낸게 아까워서   그리고 나중엔 생활자금 나오는 것 같은데  노후 생각하면 유지할지.. 해지할지 정말 정말 고민입니다.     어떻게 하는게 좋을까요....?  꼭 도움 좀 부탁드리겠습니다.  미리 감사드립니다. ^^ 
Saved at/Users/maeng-dev/Documents/

만 33세 종신보험 추천 부탁드립니다.  현재 저축형? 종신보험은 하나 가입되어있구요 종신보험 하나를 더 들려구해요 제가 보험을 검토해보니 제가 자전거를 많이 타는데 자전거 상해? 또는 대중교통시 발생하는 상해쪽과 주계약?인 사망시 보험금이 타 보험 비율대비 적은걸로 나왔어요 설계사분 계시면 연락주세요 * insurance 에서 게시물 이동되었습니다 (2018-11-10 15:25 / no : 71999)
http://www.ppomppu.co.kr/zboard/view.php?id=consulting&page=1&divpage=11&category=1&no=74118
보험가입문의  [안내사항] ① 설계와 가입문의는 아래 URL의 인증컨설턴트 이미지를 클릭하여 문의 하시기 바랍니다. - 관련 URL: http://www.ppomppu.co.kr/zboard/view.php?id=consulting&no=2509 ② 빠른 설계와 가입문의를 원하시는 회원님들께서는 본게시판의 상담신청 배너를 이용 하시기 바랍니다. - 게시글의 열람은 누구나 가능하기 때문에 개인정보는 절대 기재하지 마십시요. (이름/주민번호/연락처/이메일등) ------------------------------------------<게시물 작성 시 위 내용은 삭제하고 작성하세요!>--------------------------------------------   대략적인 개인정보는 아래와 같습니다. 1. 나이 : 29살 2. 성별 : 남자 3. 병원기록 : 입원, 수술은 한적 없으나, 허리아파서 병원치료 받은적 있습니다.                  건강검진결과 이상없음   현재 가지고 있는 보험은 아래 이미지의 보험 1개입니다. 주위에서 어린이보험을 추천해서 가입하려고 알아보는중입니다. 아래 보험을 참고했을때, 어린이보험이나 기타 다른 보험은 어떤걸 가입하는 것이 좋을까요?      
http://www.ppomppu.co.kr/zboard/view.php?id=consulting&page=1

KeyboardInterrupt: 